In [34]:
import os
import re
import json
import pandas as pd
import argparse
from openai import OpenAI
from dotenv import load_dotenv


In [35]:
def get_calendar_data(DATA_PATH):
    return pd.read_csv(DATA_PATH + "calendar_data.csv")


def get_question(DATA_PATH):
    with open(DATA_PATH + "question.json") as json_file:
        json_data = json.load(json_file)
    return json_data
    

    
def get_prompt(question, examples):
    
    task = f"""{question}
    """

    PROMPT = f"""Answer the following question based on the JSON file with agenda from Google Calendar.\n Question: {task}'
                \n\nJSON file with agenda:\n{examples}. Today's date is '2024-04-02 09:02:30' with date format '%Y-%m-%d %H:%M:%S'"""
                        
    return PROMPT


In [36]:

def get_completion(MODEL, PROMPT):

    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content":PROMPT,
            }
        ],
        model=MODEL,
    )

    # verify the output
    return chat_completion.choices[0].message.content

# Baseline test for GPT-4

In [ ]:

DATA_PATH = "../data/"
MODEL = "gpt-4"
# MODEL = "ft:gpt-3.5-turbo-1106:ninjatech-ai-dev::8sQyzUKb"
print(MODEL)
    
result = {'success': 0, 'wrong_answer': 0, 'error': 0}
question_answer_summary = {'question': '',
                            'true_answer':'', 
                            'generated_answer': ''
                            }
result_list = []
    
calendar_data = get_calendar_data(DATA_PATH=DATA_PATH)
queston_answer_pairs = get_question(DATA_PATH=DATA_PATH)
calendar_data_JSON = calendar_data.to_json() 
    
ground_true = []
for i in range(5):
        
    result = {'success': 0, 'wrong_answer': 0, 'error': 0}
        
    for q_a_pair in queston_answer_pairs:

        # print(f"""Question: {q_a_pair['question']}\n True answer: {q_a_pair['answer']}""")
            
        PROMPT = get_prompt(q_a_pair['question'], calendar_data_JSON)
            
        llm_reply = get_completion(MODEL, PROMPT)
        # print(f"""The answer of LLM: {llm_reply}""")
            
        if str(q_a_pair['answer']) in llm_reply.lower():
            result['success'] += 1
        else:
            result['wrong_answer'] += 1
            
        # result analysis
        question_answer_summary['question'] = q_a_pair['question']
        question_answer_summary['true_answer'] = q_a_pair['answer']
        question_answer_summary['generated_code'] = llm_reply
    print(result)
    result_list.append(result.copy())
print(result_list)

# with open('baseline_' + f"""{MODEL}""" + '_results.json', 'w', encoding='utf-8') as f:
#     json.dump(result_list, f, indent=4) 

print('End')


# Baseline - test the GPT3.5 finetuned

In [40]:
from datetime import datetime
from pytz import timezone

DATA_PATH = "../data/"
# MODEL = "gpt-4"
MODEL = "ft:gpt-3.5-turbo-1106:ninjatech-ai-dev::8sQyzUKb"
print(MODEL)

calendar_data = get_calendar_data(DATA_PATH=DATA_PATH)
queston_answer_pairs = get_question(DATA_PATH=DATA_PATH)
calendar_data_JSON = calendar_data.to_json() 

# convert the start and end time into datetime format (%Y-%m-%d %H:%M:%S)
def convert_date(calendar_data):   
    # Convert 'start' and 'end' columns to datetime objects
    calendar_data['start'] = pd.to_datetime(calendar_data['start'])
    calendar_data['end']   = pd.to_datetime(calendar_data['end'])

    # Format 'start' and 'end' columns into the desired string format
    calendar_data['start'] = calendar_data['start'].dt.strftime('%Y-%m-%d %H:%M:%S')
    calendar_data['end']   = calendar_data['end'].dt.strftime('%Y-%m-%d %H:%M:%S')

    return calendar_data
calendar_data = convert_date(calendar_data)

ft:gpt-3.5-turbo-1106:ninjatech-ai-dev::8sQyzUKb


In [51]:
import pytz

def convert_dataframe_to_agenda(calendar_data):
    agenda_type = {}
    agenda_type["recurrent"] = False
    agenda_type["summary"] = calendar_data["summary"]
    agenda_type["description"] = None
    agenda_type["attendees"] = calendar_data["attendees"]
    agenda_type["original_timezone"] = pytz.timezone('Australia/Sydney')
    agenda_type["start"] = calendar_data["start"]
    agenda_type["end"] = calendar_data["end"]
    agenda_type["duration"] = calendar_data["duration"]
    
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    weekday = weekdays[pd.to_datetime(calendar_data["start"], format='%Y-%m-%d %H:%M:%S').weekday()]
    
    agenda_type["weekday"] = weekday
    
    return agenda_type

calendar_data_to_agenda = []
for i in range(len(calendar_data)):
    calendar_data_to_agenda.append(convert_dataframe_to_agenda(calendar_data.iloc[i]))
print(calendar_data_to_agenda[2])

def get_vinalla_prompt(question, examples):
    
    task = f"""{question}
    """

    PROMPT = f"""Answer the following question based on the JSON file with agenda from Google Calendar.\n Question: {task}'
                \n\nJSON file with agenda:\n{examples}."""
                        
    return PROMPT


{'recurrent': False, 'summary': 'Team Introduction', 'description': None, 'attendees': 'walter@ninjatech.ai(accepted)\\n arash@ninjatech.ai(accepted)', 'original_timezone': <DstTzInfo 'Australia/Sydney' LMT+10:05:00 STD>, 'start': '2024-02-15 09:00:00', 'end': '2024-02-15 09:30:00', 'duration': 1800, 'weekday': 'Thursday'}


In [52]:

result = {'success': 0, 'wrong_answer': 0, 'error': 0}
question_answer_summary = {'question': '',
                            'true_answer':'', 
                            'generated_answer': ''
                            }
result_list = []
    
ground_true = []
for i in range(5):
        
    result = {'success': 0, 'wrong_answer': 0, 'error': 0}
        
    for q_a_pair in queston_answer_pairs:

        # print(f"""Question: {q_a_pair['question']}\n True answer: {q_a_pair['answer']}""")
            
        PROMPT = get_vinalla_prompt(q_a_pair['question'], calendar_data_to_agenda)
            
        llm_reply = get_completion(MODEL, PROMPT)
        # print(f"""The answer of LLM: {llm_reply}""")
            
        if str(q_a_pair['answer']) in llm_reply.lower():
            result['success'] += 1
        else:
            result['wrong_answer'] += 1
            
        # result analysis
        question_answer_summary['question'] = q_a_pair['question']
        question_answer_summary['true_answer'] = q_a_pair['answer']
        question_answer_summary['generated_code'] = llm_reply
    print(result)
    result_list.append(result.copy())
print(result_list)

# with open('baseline_' + f"""{MODEL}""" + '_results.json', 'w', encoding='utf-8') as f:
#     json.dump(result_list, f, indent=4) 

print('End')

{'success': 8, 'wrong_answer': 38, 'error': 0}
{'success': 7, 'wrong_answer': 39, 'error': 0}
{'success': 4, 'wrong_answer': 42, 'error': 0}
{'success': 10, 'wrong_answer': 36, 'error': 0}
{'success': 8, 'wrong_answer': 38, 'error': 0}
[{'success': 8, 'wrong_answer': 38, 'error': 0}, {'success': 7, 'wrong_answer': 39, 'error': 0}, {'success': 4, 'wrong_answer': 42, 'error': 0}, {'success': 10, 'wrong_answer': 36, 'error': 0}, {'success': 8, 'wrong_answer': 38, 'error': 0}]
End
